<a href="https://colab.research.google.com/github/pavankalyano76/PavanKalyan_INFO5731_Spring2025/blob/main/Natukula_Pavan_Kalyan_Assignment02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import json
import re
import pandas as pd

pages = 41
dict = {}
narr_count = 0
#main_url = "https://ddr.densho.org/narrators/?page={}"
for i in range(1,pages+1):

    main_url = f'https://ddr.densho.org/narrators/?page={i}'
    link1 = Request(main_url, headers={'User-Agent': 'Mozilla/5.0'})
    url1 = urlopen(link1)

    data1 = url1.read()
    #data1_soup = BeautifulSoup(data1)

    soup = BeautifulSoup(data1,"html.parser")
    #print(type(soup))

    div_clause = soup.find(id = "list_tab")


    narrators = div_clause.find_all(class_='media-body')
    #print(narrators)

    for i in narrators:
        if len(dict) == 904:
            break
        tags=i.find("a")
        tag2 = i.find(class_ = "source muted")
        name = tags.get_text()
        details = tag2.get_text().strip()
        dict[name] = details
        narr_count+=1

    if len(dict) == 904:
        break
#print(dict)
df = pd.DataFrame(list(dict.items()), columns=["Name", "Details"])
#df = pd.DataFrame(dict, columns=["Name", "Details"])
df

df.to_csv("Narrator Information.csv",index=False)


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
# Write code for each of the sub parts with proper comments.
import pandas as pd
df = pd.read_csv('Narrator Information.csv')
df = df.dropna()

#1. Removing Noise
df['CleanedName'] = df['Name'].str.replace('[^\w\s]','',regex=True)
df['CleanedDetails'] = df['Details'].str.replace('[^\w\s]','',regex=True)

#2. Removing Numbers
df['CleanedName'] = df['CleanedName'].str.replace('\d+', '', regex=True)
df['CleanedDetails'] = df['CleanedDetails'].str.replace('\d+', '', regex=True)

#3. Removing Stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

df['CleanedName'] = df['CleanedName'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['CleanedDetails'] = df['CleanedDetails'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

#4. Lowercase all texts
df['CleanedName'] = df['CleanedName'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['CleanedDetails'] = df['CleanedDetails'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#5. Stemming
from nltk.stem import PorterStemmer
st = PorterStemmer()
df['CleanedName']=df['CleanedName'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
df['CleanedDetails']=df['CleanedDetails'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))


#6. Lemmatization
from textblob import Word
import nltk
nltk.download('wordnet')

df['CleanedName'] = df['CleanedName'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
df['CleanedDetails'] = df['CleanedDetails'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

df.head()
df.to_csv('Narrators_Information_Cleaned.csv', index=False)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
########################### PART I ##################################
import pandas as pd
import nltk
from nltk import pos_tag, word_tokenize
from collections import Counter

# Download necessary NLTK resources
nltk.download("punkt")
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt_tab') # Download the punkt_tab data for sentence tokenization

# Load CSV file

df_pos= pd.read_csv('/content/Narrators_Information_Cleaned.csv')

# UDF for tagging and countings pos
def pos_tagging(text):
    tokens = word_tokenize(str(text))  # Tokenize the text
    tagged_words = pos_tag(tokens)  # POS tagging

    # Counter is a class in collection that creates a dictionary with pos tag and count in partucular text
    pos_counts_dict = Counter(tag for word, tag in tagged_words)

    # Map NLTK POS tags to simplified categories
    countOfNouns = sum(pos_counts_dict[tag] for tag in ["NN", "NNS", "NNP", "NNPS"])  # Nouns
    countOfVerbs = sum(pos_counts_dict[tag] for tag in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"])  # Verbs
    countOfAdjectives = sum(pos_counts_dict[tag] for tag in ["JJ", "JJR", "JJS"])  # Adjectives
    countOfAdverbs = sum(pos_counts_dict[tag] for tag in ["RB", "RBR", "RBS"])  # Adverbs

    return countOfNouns, countOfVerbs, countOfAdjectives, countOfAdverbs

# Apply pos tagging function to "CleanedDetails" column
df_pos[['Nouns', 'Verbs', 'Adjectives', 'Adverbs']] = pd.DataFrame(df_pos["CleanedDetails"].apply(pos_tagging).to_list(), index=df_pos.index)


# Sum total occurrences across all rows
total_nouns = df_pos["Nouns"].sum()
total_verbs = df_pos["Verbs"].sum()
total_adjectives = df_pos["Adjectives"].sum()
total_adverbs = df_pos["Adverbs"].sum()

# Output the results
print(f"Total Nouns: {total_nouns}")
print(f"Total Verbs: {total_verbs}")
print(f"Total Adjectives: {total_adjectives}")
print(f"Total Adverbs: {total_adverbs}")





[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Total Nouns: 8896
Total Verbs: 2060
Total Adjectives: 2733
Total Adverbs: 466


In [ ]:
import pandas as pd
import spacy
import benepar
from spacy import displacy
from nltk import Tree


# Load spaCy model and benepar
nlp = spacy.load("en_core_web_sm")
benepar.download('benepar_en3')
nlp.add_pipe("benepar", config={"model": "benepar_en3"})


def plot_dependency_tree(doc):
    print("\nDependency Parsing Tree (Text Representation):")
    for token in doc:
        print(f"{token.text} --({token.dep_} → {spacy.explain(token.dep_)})--> {token.head.text}")

    # Visual dependency tree using displaCy
    displacy.render(doc, style='dep', jupyter=True, options={'compact': True, 'distance': 90})


def plot_constituency_tree(doc):
    for sent in doc.sents:
        print("\nConstituency Parsing Tree (Text Representation):")
        print(sent._.parse_string)

        # Visual constituency tree with nltk
        tree = Tree.fromstring(sent._.parse_string)
        tree.pretty_print()



sentence = df_pos['CleanedDetails'][0]
doc = nlp(sentence)

print(f"Sentence: {sentence}")

plot_dependency_tree(doc)
plot_constituency_tree(doc)


[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!
/usr/local/lib/python3.11/dist-packages/benepar/parse_chart.py:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue o

Sentence: nisei femal born may selleck washington spent much childhood beaverton oregon father own farm influenc earli

Dependency Parsing Tree (Text Representation):
nisei --(compound → compound)--> femal
femal --(nsubj → nominal subject)--> selleck
born --(acl → clausal modifier of noun (adjectival clause))--> femal
may --(aux → auxiliary)--> selleck
selleck --(compound → compound)--> washington
washington --(nsubj → nominal subject)--> spent
spent --(ROOT → root)--> spent
much --(amod → adjectival modifier)--> beaverton
childhood --(compound → compound)--> beaverton
beaverton --(dobj → direct object)--> spent
oregon --(compound → compound)--> father
father --(appos → appositional modifier)--> beaverton
own --(amod → adjectival modifier)--> earli
farm --(compound → compound)--> influenc
influenc --(compound → compound)--> earli
earli --(npadvmod → noun phrase as adverbial modifier)--> spent


/usr/local/lib/python3.11/dist-packages/torch/distributions/distribution.py:56: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(



Constituency Parsing Tree (Text Representation):
(S (NP (NP (FW nisei) (NN femal)) (VP (VP (VBN born) (MD may) (RB selleck)) (NNP washington))) (VP (VBD spent) (NP (JJ much) (NN childhood)) (NP (NP (UCP (NN beaverton) (FW oregon)) (NN father)) (JJ own) (NN farm) (FW influenc) (FW earli))))
                                                       S                                                                    
                  _____________________________________|_________________________________                                    
                 |                                                                       VP                                 
                 |                                 ______________________________________|____________                       
                 NP                               |         |                                         NP                    
        _________|________                        |         |                    

In [ ]:
########################### PART III ##################################
import spacy
import pandas as pd
from collections import Counter

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

# Load the CSV file
df_NER = pd.read_csv('/content/Narrators_Information_Cleaned.csv')

# Initialize counters and entity storage
entity_counter = Counter()
entity_examples = {"PERSON": [], "ORG": [], "GPE": [], "PRODUCT": [], "DATE": []}

# Extract entities from the "CleanedDetails" column
for i in df["CleanedDetails"]:
    doc = nlp(i)
    for ent in doc.ents:
        if ent.label_ in entity_examples:
            entity_counter[ent.label_] += 1
            #entity_examples[ent.label_].append(ent.text)



# Print results
print('Named Entity Recognition Results are: ')
for entity, count in entity_counter.items():
    print(f"{entity}: {count} Entities")



Named Entity Recognition Results are: 
PERSON: 777 Entities
GPE: 1523 Entities
DATE: 237 Entities
ORG: 238 Entities
PRODUCT: 1 Entities


# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

In [ ]:
#################### Part-1 ###################
import time
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.error import HTTPError
import json
import re
import pandas as pd
import csv

NoOfPages = 55
Dlist = []
with open('github_marketplace_products1.csv', mode='w', newline='', encoding='utf-8') as file:
  writer = csv.DictWriter(file, fieldnames=['Product Name', 'Description', 'URL', 'Page Number'])
  writer.writeheader()
  for j in range(1,NoOfPages+1):
    Git_URL = f"https://github.com/marketplace?page={j}&type=actions"

    GitLink = Request(Git_URL, headers={'User-Agent': 'Mozilla/5.0'})
    GitURL = urlopen(GitLink)

    GitResponse = GitURL.read()
      #data1_soup = BeautifulSoup(data1)

    GitSoup = BeautifulSoup(GitResponse,"html.parser")
    #print(soup.prettify())
    #Action_Div = soup.find_all(class_='flex-1')
    #print(type(Action_Div))

    Action_Div = GitSoup.find_all(class_="position-relative border rounded-2 d-flex marketplace-common-module__marketplace-item--MohVH gap-3 p-3")
    #print(Action_Div)
    for i in Action_Div:
      name_tag = i.find(class_="marketplace-common-module__marketplace-item-link--jrIHf line-clamp-1")
      productName = name_tag.get_text().strip()
      desc_tag = i.find('p')
      productDescription = desc_tag.get_text().strip()
      URL_tag = name_tag['href']
      ProductURL = 'https://github.com/'+str(URL_tag)
      writer.writerow({
                'Product Name': productName,
                'Description': productDescription,
                'URL': ProductURL,
                'Page Number': j
            })
      time.sleep(0.2)





In [ ]:

###################### part2 ######################
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re


df_git = pd.read_csv('github_marketplace_products1.csv')

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
###################  PREPROCESSING ##########################
# Initialize Lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# function to do all preprocessing
def preprocess_text(text):

    if not isinstance(text, str):
        text = str(text)  # Convert non-string types to string

    # Convert to lowercase
    text = text.lower()

    # Remove special characters and digits (keeping only letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and apply lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    return ' '.join(tokens)

# Apply preprocessing on 'Description' column
df_git['Description'] = df_git['Description'].apply(preprocess_text)

###################  DATA QUALITY CHECKS ##########################


# 1. Check for missing values
missing_data = df_git.isnull().sum()


# 2. Remove duplicates based on 'Product Name' and 'URL' columns
df_git = df_git.drop_duplicates(subset=['Product Name', 'URL'])

# 3. Ensure that 'Product Name' and 'URL' are not empty
df_git = df_git[df_git['Product Name'].notna() & df_git['URL'].notna()]

# 4. Check for invalid URLs (basic check)
url_pattern = re.compile(r'^(https?://)?(www\.)?github\.com/.+')
df_git['Valid_URL'] = df_git['URL'].apply(lambda x: bool(url_pattern.match(x)))



# Filter out rows with invalid URLs
df_git = df_git[df_git['Valid_URL']]


# Save the cleaned data to a new CSV file
df_git.to_csv('cleaned_github_marketplace_products.csv', index=False)


if missing_data.any():
    print("There is missing data.")
else:
    print("No missing data.")



print("\nCleaned Data Sample:")
print(df_git.head())




No missing data.

Cleaned Data Sample:
                   Product Name  \
0                TruffleHog OSS   
1                 Metrics embed   
2  yq - portable yaml processor   
3                  Super-Linter   
4        Gosec Security Checker   

                                         Description  \
0                      scan github action trufflehog   
1  infographics generator plugins option display ...   
2      create read update delete merge validate yaml   
3  superlinter readytorun collection linters code...   
4                         run gosec security checker   

                                                 URL  Page Number  Valid_URL  
0  https://github.com//marketplace/actions/truffl...            1       True  
1  https://github.com//marketplace/actions/metric...            1       True  
2  https://github.com//marketplace/actions/yq-por...            1       True  
3  https://github.com//marketplace/actions/super-...            1       True  
4  https://github.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [ ]:
############################# PART I #################################################
import tweepy

# Set your keys and tokens here
api_key = 'HLWL67RpwHb5ZF450wNnjHGuE'
api_key_secret = 'gzvolj1ISwrCXCbCwDYEZIozsGQHlC3tbQhh5vF4hInSqE95jq'
access_token = '1892332327445102592-LKi5zuQKTyxUjHGg9NBtnPBFZRUZmL'
access_token_secret = 'XZaaPy3WpYTgMLPxWLjoa3njWp6mU3dlI6t62wlzBaF5k'

# Authenticate with Twitter
auth = tweepy.OAuth1UserHandler(
    consumer_key=api_key,
    consumer_secret=api_key_secret,
    access_token=access_token,
    access_token_secret=access_token_secret
)
api = tweepy.API(auth)
max_tweets = 1000
hashtag = '#generativeAI -is:retweet'

client = tweepy.Client(bearer_token='AAAAAAAAAAAAAAAAAAAAADagzQEAAAAAg1n8bETsGPF1BGuI7rcaKzQgEFg%3D1wm1GnUryrzjA6gg6ulcEMHeW3sIS2EoLop2TgrJGhduzffO1d')
tweets = client.search_recent_tweets(query=hashtag, tweet_fields=["created_at", "text", "author_id"], max_results=100)




In [ ]:
#print(tweets)
# Create a dictionary to store tweet data
tweetDict= {'tweet_id': [],'user_id': [],'tweet_time': [],'tweetText': []}

if tweets.data:
    for i in tweets.data:
        tweetDict['tweet_id'].append(i.id)
        tweetDict['user_id'].append(i.author_id)
        tweetDict['tweet_time'].append(i.created_at)
        tweetDict['tweetText'].append(i.text)

# dataframe creation
dfTweets = pd.DataFrame(tweetDict)

# Display the DataFrame
print(dfTweets.head())

# Save DataFrame to a CSV file
dfTweets.to_csv("Generative_AI_Tweets.csv", index=False)

              tweet_id              user_id                tweet_time  \
0  1892333447869317230  1637311944008044544 2025-02-19 22:00:26+00:00   
1  1892333446673928615             50065957 2025-02-19 22:00:26+00:00   
2  1892333426964881545  1829231426673266688 2025-02-19 22:00:21+00:00   
3  1892333322162098357  1219122060531658754 2025-02-19 21:59:56+00:00   
4  1892333208886259740  1724183495122079744 2025-02-19 21:59:29+00:00   

                                           tweetText  
0  Image FX 自由研究その199\n[フェンシング女子🥰]\n\nCompetitive...  
1  How "responsible" are your org's marketing pra...  
2  Connected Intelligence, Boomi Locks Down APIs ...  
3  Fascinated by the fusion of art and technology...  
4  Microsoft touts Muse, a new generative AI mode...  


In [ ]:
######################### Part II ##########################

import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re


df_Tweet = pd.read_csv('/content/Generative_AI_Tweets.csv')
#print(df_Tweet.describe)

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
###################  PREPROCESSING ##########################
# Initialize Lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# function to do all preprocessing
def preprocess_text_1(text):

    if not isinstance(text, str):
        text = str(text)  # Convert non-string types to string

    # Convert to lowercase
    text = text.lower()

    # Remove special characters and digits (keeping only letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords and apply lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    return ' '.join(tokens)

# Apply preprocessing on 'Description' column
df_Tweet['tweetText'] = df_Tweet['tweetText'].apply(preprocess_text_1)

###################  DATA QUALITY CHECKS ##########################


# 1. Check for missing values
missing_data_1 = df_Tweet.isnull().sum()

# 2. Remove duplicates
df_Tweet = df_Tweet.drop_duplicates(subset=['tweetText', 'user_id'])

# Save the cleaned data to a new CSV file
df_Tweet.to_csv('cleaned_Generative_AI_Tweets.csv', index=False)


if missing_data_1.any():
    print("There is missing data.")
    df_Tweet['tweetText'].fillna('Unknown', inplace=True)
    df_Tweet['user_id'].fillna('Unknown', inplace=True)
    df_Tweet['author_id'].fillna('No information available',inplace=True)

else:
    print("No missing data.")



print("\nCleaned Data Sample:")
print(df_Tweet.head())




No missing data.

Cleaned Data Sample:
              tweet_id              user_id                 tweet_time  \
0  1892333447869317230  1637311944008044544  2025-02-19 22:00:26+00:00   
1  1892333446673928615             50065957  2025-02-19 22:00:26+00:00   
2  1892333426964881545  1829231426673266688  2025-02-19 22:00:21+00:00   
3  1892333322162098357  1219122060531658754  2025-02-19 21:59:56+00:00   
4  1892333208886259740  1724183495122079744  2025-02-19 21:59:29+00:00   

                                           tweetText  
0  image fx competitive fencing tournament ai aia...  
1  responsible orgs marketing practice true compe...  
2  connected intelligence boomi lock apis generat...  
3  fascinated fusion art technology explore new g...  
4  microsoft tout muse new generative ai model vi...  


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

Assignment was good but it was hard. The good thing is this assignment helps to learn how to webscrap any website.

# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog